# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902035


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:27,  3.00s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:09,  2.50s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:46,  1.78s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:30,  1.28s/it]

...

Rendering models:  77%|███████▋  | 24/31 [00:09<00:01,  6.12it/s]

Rendering models:  84%|████████▍ | 26/31 [00:09<00:00,  6.77it/s]

Rendering models:  90%|█████████ | 28/31 [00:09<00:00,  7.55it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  8.10it/s]

equidad1                              0.135466
Tommy80452                            0.001309
not-logged-in-6ab0e00fa5a467f5f742    0.017123
Mouring                               0.001086
klmasters                             0.000689
not-logged-in-cae133427d99aac1e554    0.005381
not-logged-in-8c0c44faf754d344b18a    0.000640
Thedunkmasta                          0.007642
Space1000                             0.031491
irwansyah                             0.000498
sinjinza1984                          0.019430
not-logged-in-ab1612bc18cd7aab85db    0.000322
not-logged-in-f2eb212bec3c0ec6797b    0.036752
coyotes54                             0.001002
mabaker                               0.028331
mariafarhat                           0.004158
galaxypotato                          0.000972
JoseA.G.v                             0.002429
TheEpicPrimius123                     0.002048
not-logged-in-44beefa7ea1fce864d18    0.000749
sepsamadi                             0.001587
Planetme     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())